# 模型参数的访问、初始化和共享

在[“线性回归的简洁实现”](../chapter_deep-learning-basics/linear-regression-gluon.ipynb)一节中，我们通过`init`模块来初始化模型的全部参数。我们也介绍了访问模型参数的简单方法。本节将深入讲解如何访问和初始化模型参数，以及如何在多个层之间共享同一份模型参数。

我们先定义一个与上一节中相同的含单隐藏层的多层感知机。我们依然使用默认方式初始化它的参数，并做一次前向计算。与之前不同的是，在这里我们使用torch.nn中的`init`模块，它包含了多种模型初始化方法。

In [1]:
import torch
from torch import nn

net = nn.Sequential()
net.add_module("hidden", nn.Linear(20, 256))
net.add_module("activation", nn.ReLU())
net.add_module("output", nn.Linear(256, 10))

X = torch.rand(2, 20)
Y = net(X) # 前向计算

## 访问模型参数

对于使用`Sequential`类构造的神经网络，我们可以通过方括号`[]`来访问网络的任一层。回忆一下上一节中提到的`Sequential`类与`Module`类的继承关系。对于`Sequential`实例中含模型参数的层，我们可以通过`Module`类的`parameters()`或者`named_parameters()`函数来访问该层包含的所有参数。下面，访问多层感知机`net`中隐藏层的所有参数。

In [2]:
for name, param in net.named_parameters():
    print(name, param)
    
type(net.named_parameters())

hidden.weight Parameter containing:
tensor([[-0.2020, -0.0940, -0.1824,  ..., -0.0068,  0.0970,  0.1482],
        [ 0.0019,  0.2137,  0.1696,  ..., -0.0510, -0.1160,  0.2125],
        [ 0.1995,  0.1376,  0.1833,  ..., -0.1808, -0.2169,  0.1999],
        ...,
        [-0.0865, -0.0967,  0.1390,  ...,  0.1609, -0.2124,  0.2044],
        [ 0.0271, -0.0312, -0.0570,  ..., -0.2049,  0.0278,  0.0193],
        [ 0.0395,  0.0134, -0.1902,  ..., -0.1404, -0.0844,  0.1393]],
       requires_grad=True)
hidden.bias Parameter containing:
tensor([ 7.8103e-03,  8.7859e-02,  3.5132e-02, -1.6534e-01, -1.2234e-01,
        -1.5377e-01, -1.3543e-01,  4.0223e-02, -2.0627e-01,  5.2887e-02,
         1.9170e-01,  3.1859e-02,  1.7611e-01, -4.6360e-02, -6.4578e-02,
        -2.1074e-01,  1.5730e-01,  1.6702e-01, -3.8143e-02,  1.2716e-01,
        -8.1479e-02,  1.7518e-01, -8.7225e-02,  5.9440e-02,  3.2729e-02,
        -1.9102e-01, -6.6407e-02,  5.4632e-02, -1.1433e-02, -1.9550e-01,
        -2.1350e-01,  1.9242e-0

generator

可以看到，我们得到了一个返回参数名和参数值的迭代器。其中隐藏层权重参数的名称为`hidden.weight`，它由`net[0]`的名称（`hidden`）和自己的变量名（`weight`）组成。


**注：如果单独调用`net[0].named_parameters()`获得的权重参数名为`weight`**


为了访问特定参数，我们可以使用`net.state_dict()`来获得一个由参数名映射到参数值的字典（类型为`OrderedDict`）。通过名字来访问字典里的元素，也可以直接使用它的变量名。下面两种方法是等价的，但通常后者的代码可读性更好。

In [3]:
net[0].state_dict()['weight'], net[0].weight

(tensor([[-0.2020, -0.0940, -0.1824,  ..., -0.0068,  0.0970,  0.1482],
         [ 0.0019,  0.2137,  0.1696,  ..., -0.0510, -0.1160,  0.2125],
         [ 0.1995,  0.1376,  0.1833,  ..., -0.1808, -0.2169,  0.1999],
         ...,
         [-0.0865, -0.0967,  0.1390,  ...,  0.1609, -0.2124,  0.2044],
         [ 0.0271, -0.0312, -0.0570,  ..., -0.2049,  0.0278,  0.0193],
         [ 0.0395,  0.0134, -0.1902,  ..., -0.1404, -0.0844,  0.1393]]),
 Parameter containing:
 tensor([[-0.2020, -0.0940, -0.1824,  ..., -0.0068,  0.0970,  0.1482],
         [ 0.0019,  0.2137,  0.1696,  ..., -0.0510, -0.1160,  0.2125],
         [ 0.1995,  0.1376,  0.1833,  ..., -0.1808, -0.2169,  0.1999],
         ...,
         [-0.0865, -0.0967,  0.1390,  ...,  0.1609, -0.2124,  0.2044],
         [ 0.0271, -0.0312, -0.0570,  ..., -0.2049,  0.0278,  0.0193],
         [ 0.0395,  0.0134, -0.1902,  ..., -0.1404, -0.0844,  0.1393]],
        requires_grad=True))

权重梯度的形状和权重的形状一样。因为我们还没有进行反向传播计算，所以梯度为None。

In [4]:
net[0].weight.grad == None

True

类似地，我们可以访问其他层的参数，如输出层的偏差值。

In [5]:
net[2].bias

Parameter containing:
tensor([ 0.0053, -0.0368, -0.0521, -0.0278, -0.0111,  0.0378,  0.0265, -0.0141,
        -0.0245,  0.0381], requires_grad=True)

## 初始化模型参数

我们在[“数值稳定性和模型初始化”](../chapter_deep-learning-basics/numerical-stability-and-init.ipynb)一节中描述了模型的默认初始化方法：权重参数元素为[-0.07, 0.07]之间均匀分布的随机数，偏差参数则全为0。但我们经常需要使用其他方法来初始化权重。PyTorch在`nn.init`模块里提供了多种预设的初始化单个参数的方法。在下面的例子中，我们将隐藏层的权重参数初始化成均值为0、标准差为0.01的正态分布随机数，并将偏差参数初始化为常数0。

In [6]:
nn.init.normal_(net[0].weight, mean=0, std=0.01)  # weight
nn.init.constant_(net[0].bias, 0) # bias

net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0113, -0.0137,  0.0041,  0.0029, -0.0041,  0.0011, -0.0069,  0.0024,
          0.0012, -0.0144,  0.0010, -0.0104, -0.0050,  0.0058,  0.0140, -0.0094,
          0.0044,  0.0118,  0.0015,  0.0139]), tensor(0.))

## 自定义初始化方法

如果需要将模型中所有网络层的参数按照相同的策略进行初始化，可以自定义一个初始化方法（如`weight_init`），然后使用`.apply(weight_init)`进行自定义初始化。在下面的例子里，我们令Linear层的权重有一半概率初始化为0，有另一半概率初始化为$[-10,-5]$和$[5,10]$两个区间里均匀分布的随机数。


**注：参考自：<https://discuss.pytorch.org/t/reset-the-parameters-of-a-model/29839>**

In [7]:
# PyTorch不允许对requires_gead=True的tensor做inplace操作。
# 所以需要取出weight的 data 属性，对它进行相应的处理
def weight_init(m):
    if isinstance(m, nn.Linear):
        print('Init', m.weight.shape)
        m.weight.data.uniform_(-10, to=10)
        m.weight.data *= (m.weight.data.abs() >= 5).float()

net.apply(weight_init)
net[0].weight.data[0]

Init torch.Size([256, 20])
Init torch.Size([10, 256])


tensor([-5.5105,  8.5852,  0.0000, -5.6937,  0.0000,  0.0000, -5.8642,  6.8767,
        -0.0000, -0.0000, -0.0000, -7.2237, -0.0000,  0.0000, -7.3165,  6.4280,
        -0.0000,  8.8226, -0.0000, -0.0000])

**注：因为 PyTorch 并没有提供对模型所有参数进行初始化的方法，我们写一个自定义的全局初始化参数的方法加入到 d2ltorch 包中。(该方法来自 [Weights-Initializer-pytorch](https://github.com/3ammor/Weights-Initializer-pytorch) )**

In [8]:
def params_init(model, init, **kwargs): # 本方法已保存在d2ltorch包中方便以后使用

    def initializer(m):
        if isinstance(m, nn.Conv2d):
            init(m.weight.data, **kwargs)
            m.bias.data.fill_(0)

        elif isinstance(m, nn.Linear):
            init(m.weight.data, **kwargs)
            m.bias.data.fill_(0)

        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1.0)
            m.bias.data.fill_(0)

        elif isinstance(m, nn.BatchNorm1d):
            m.weight.data.fill_(1.0)
            m.bias.data.fill_(0)

    model.apply(initializer)

此外，我们还可以通过`Parameter`类的`data`属性来直接改写模型参数。例如，在下例中我们将隐藏层参数在现有的基础上加1。

In [9]:
net[0].weight.data = net[0].weight.data + 1
net[0].weight.data[0]

tensor([-4.5105,  9.5852,  1.0000, -4.6937,  1.0000,  1.0000, -4.8642,  7.8767,
         1.0000,  1.0000,  1.0000, -6.2237,  1.0000,  1.0000, -6.3165,  7.4280,
         1.0000,  9.8226,  1.0000,  1.0000])

## 共享模型参数

在有些情况下，我们希望在多个层之间共享模型参数。[“模型构造”](model-construction.ipynb)一节介绍了如何在`Module`类的`forward`函数里多次调用同一个层来计算。

**注：经过查找，除了上述方法外，PyTorch并没有提供其他的共享参数并且传播时保持梯度的方法。**

## 小结

* 有多种方法来访问、初始化和共享模型参数。
* 可以自定义初始化方法。


## 练习

* 查阅有关`nn.init`模块的PyTorch文档，了解不同的参数初始化方法。
* 尝试在`net`实例化后、`net(X)`前访问模型参数，观察模型参数的形状。
* 构造一个含共享参数层的多层感知机并训练。在训练过程中，观察每一层的模型参数和梯度。



## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/987)

![](../img/qr_parameters.svg)